# Notebook lecture 13: MPC for Drone Flight
&copy; 2025 ETH Zurich, Joël Gmür, Joël Lauper, Niclas Scheuer, Dejan Milojevic; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

Authors:
- Joël Gmür; jgmuer@ethz.ch
- Joël Lauper; jlauper@ethz.ch
- more (please add your name)

## Description
This week's Jupyter notebook involves creating a controller for a simple 2D representation of a drone. In particular, LQR and MPC will be compared directly.

To start, run the following cell to install the necessary modules and import the libraries.

In [1]:
!pip install numpy scipy matplotlib ipywidgets control cvxpy


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets
import control as ctrl
import cvxpy as cp
from scipy.linalg import expm, solve_continuous_are, solve_discrete_are
from scipy.integrate import solve_ivp
from IPython.display import display, clear_output
from typing import Tuple
from numba import njit
from tqdm.notebook import tqdm

np.set_printoptions(suppress=True, precision=3)

This week's notebook aligns with Problem Set 13, and they are intended to be solved together. 

# Exercise 1c: Continuous-Time LQR Controller

### Implement the linearized system in Python

After you have linearized the equations of motion $\dot{x} = f(x, u)$ in the problem set, please implement it in Python below:

In [3]:
# TODO: Add constants
m = 1.0 # kg
l = 0.25 # m 
I_z = 0.01 # kg*m^2
g = 9.81 # m/s^2

# TODO: Implement A and B matrices
A = np.array([[0,1,0,0,0,0],
            [0,0,0,0,g,0],
            [0,0,0,1,0,0],
            [0,0,0,0,0,0],
            [0,0,0,0,0,1],
            [0,0,0,0,0,0]])
B = np.array([[0,0],
            [0,0],
            [0,0],
            [1/m,1/m],
            [0,0],
            [l/I_z,-l/I_z]])
C = np.array([[1,0,0,0,0,0],
            [0,0,1,0,0,0],
            [0,0,0,0,1,0]])

def linearized_eom(x: np.ndarray, u: np.ndarray) -> np.ndarray:
    """
    Linearized equations of motion for a 2D quadrotor.

    Args:
        x (np.ndarray): State vector [x, xdot, y, ydot, theta, thetadot].
        u (np.ndarray): Control input vector [T1, T2].
    Returns:
        np.ndarray: State xdot vector.
    """
    return A @ x + B @ u

### Continuous-Time LQR

Next, find both the matrix $P$ and the feedback gain $K$ using the formulas for continuous-time LQR.

In [4]:
Q = np.diag([10, 1, 10, 1, 100, 10])
R = np.eye(2)

def clqr_P(A: np.ndarray, B: np.ndarray, Q: np.ndarray, R: np.ndarray) -> np.ndarray:
    """
    Solve the continuous-time LQR controller.

    Args:
        A (np.ndarray): State matrix.
        B (np.ndarray): Input matrix.
        Q (np.ndarray): State cost matrix.
        R (np.ndarray): Input cost matrix.
    Returns:
        np.ndarray: Optimal gain matrix K.
    """
    # TODO: Solve the continuous-time algebraic Riccati equation
    P = solve_continuous_are(A, B, Q, R)
    return P

def clqr_K(A: np.ndarray, B: np.ndarray, Q: np.ndarray, R: np.ndarray) -> np.ndarray:
    """
    Compute the optimal gain matrix K for the continuous-time LQR controller.

    Args:
        A (np.ndarray): State matrix.
        B (np.ndarray): Input matrix.
        Q (np.ndarray): State cost matrix.
        R (np.ndarray): Input cost matrix.
    Returns:
        np.ndarray: Optimal gain matrix K.
    """
    P = clqr_P(A, B, Q, R)
    # TODO: Implement the gain matrix K
    K = np.linalg.inv(R) @ B.T @ P
    return K

### Test Stabilization

In [ ]:
dt = 0.01
t_final = 10
steps = int(t_final / dt)
time = np.linspace(0, t_final, steps)

# Sliders for initial conditions
slider_x_0 = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='x_0')
slider_y_0 = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='y_0')
slider_theta_0 = widgets.FloatSlider(min=-0.5*np.pi, max=0.5*np.pi, step=0.1, value=0.0, description='theta_0')

# Runge-Kutta 4th order method
def rk4_step(dynamics: callable, x: float, dt: float) -> float:
    k1 = dynamics(x)
    k2 = dynamics(x + 0.5 * dt * k1)
    k3 = dynamics(x + 0.5 * dt * k2)
    k4 = dynamics(x + dt * k3)
    return x + (dt / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

def simulate_and_plot(x_0: float, y_0: float, theta_0: float) -> None:
    # Set initial conditions
    x = np.zeros((6, steps))
    x[:, 0] = [x_0, 0, y_0, 0, theta_0, 0]
    hover_vec = np.array([m*g/2, m*g/2])

    K_c = clqr_K(A, B, Q, R)

    def dynamics(xi):
        # TODO: Implement state-feedback control

        u = -K_c @ xi + hover_vec
        u = np.clip(u, 0, None)
        return linearized_eom(xi, u-hover_vec)
    
    for i in range(steps-1):
        x[:, i+1] = rk4_step(dynamics, x[:, i], dt)

    # Plotting
    fig, axes = plt.subplots(3, 1, figsize=(8, 8), sharex=True)
    axes[0].plot(time, x[0], label='x')
    axes[0].axhline(0, color='r', linestyle='--', label='x=0')
    axes[0].set_ylabel('X Position (m)')
    axes[0].set_ylim(-5, 5)
    axes[0].legend()

    axes[1].plot(time, x[2], label='y')
    axes[1].axhline(0, color='r', linestyle='--', label='y=0')
    axes[1].set_ylabel('Y Position (m)')
    axes[1].set_ylim(-5, 5)
    axes[1].legend()

    axes[2].plot(time, x[4], label='theta')
    axes[2].axhline(0, color='r', linestyle='--', label='theta=0')
    axes[2].set_ylabel('Pitch θ (rad)')
    axes[2].set_xlabel('Time (s)')
    axes[2].set_ylim(-0.5*np.pi, 0.5*np.pi)
    axes[2].legend()

    plt.tight_layout()
    plt.show()
        

# Interactive Plot
interactive_plot = widgets.interactive_output(simulate_and_plot, {
    'x_0': slider_x_0,
    'y_0': slider_y_0,
    'theta_0': slider_theta_0
})

display(widgets.VBox([slider_x_0, slider_y_0, slider_theta_0, interactive_plot]))


# Exercise 1e: Continuous-Time LQR Reference Tracking

### Reference Tracking

In exercise 1d of the Problem Set, you should have seen how to rewrite the linear system into error dynamics format. Now, this should be implemented below:

In [ ]:
dt = 0.01
t_final = 10
steps = int(t_final / dt)
time = np.linspace(0, t_final, steps)

# Sliders for initial conditions
slider_x_0 = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='x_0')
slider_y_0 = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='y_0')
slider_theta_0 = widgets.FloatSlider(min=-0.5*np.pi, max=0.5*np.pi, step=0.1, value=0.0, description='theta_0')
# Sliders for position reference
slider_x_ref = widgets.FloatSlider(min=-5, max=5, step=0.1, value=-2.0, description='x_ref')
slider_y_ref = widgets.FloatSlider(min=-5, max=5, step=0.1, value=4.0, description='y_ref')

# Runge-Kutta 4th order method
def rk4_step(dynamics: callable, x: float, dt: float) -> float:
    k1 = dynamics(x)
    k2 = dynamics(x + 0.5 * dt * k1)
    k3 = dynamics(x + 0.5 * dt * k2)
    k4 = dynamics(x + dt * k3)
    return x + (dt / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

def simulate_and_plot(x_0: float, y_0: float, theta_0: float, x_ref: float, y_ref: float) -> None:
    # Set initial conditions
    x = np.zeros((6, steps))
    x[:, 0] = [x_0, 0, y_0, 0, theta_0, 0]
    hover_vec = np.array([m*g/2, m*g/2])

    K_c = clqr_K(A, B, Q, R)

    def dynamics(xi):
        # TODO: Determine the reference and error state
        ref = np.array([x_ref, 0, y_ref, 0, 0, 0])
        err = xi - ref

        u = -K_c @ err + hover_vec
        u = np.clip(u, 0, None)
        return linearized_eom(xi, u-hover_vec)
    
    for i in range(steps-1):
        x[:, i+1] = rk4_step(dynamics, x[:, i], dt)

    # Plotting
    fig, axes = plt.subplots(3, 1, figsize=(8, 8), sharex=True)
    axes[0].plot(time, x[0], label='x')
    axes[0].axhline(x_ref, color='r', linestyle='--', label='x_ref')
    axes[0].set_ylabel('X Position (m)')
    axes[0].set_ylim(-5, 5)
    axes[0].legend()

    axes[1].plot(time, x[2], label='y')
    axes[1].axhline(y_ref, color='r', linestyle='--', label='y_ref')
    axes[1].set_ylabel('Y Position (m)')
    axes[1].set_ylim(-5, 5)
    axes[1].legend()

    axes[2].plot(time, x[4], label='theta')
    axes[2].set_ylabel('Pitch θ (rad)')
    axes[2].set_xlabel('Time (s)')
    axes[2].set_ylim(-0.5*np.pi, 0.5*np.pi)
    axes[2].legend()

    plt.tight_layout()
    plt.show()
        

# Interactive Plot
interactive_plot = widgets.interactive_output(simulate_and_plot, {
    'x_0': slider_x_0,
    'y_0': slider_y_0,
    'theta_0': slider_theta_0,
    'x_ref': slider_x_ref,
    'y_ref': slider_y_ref,
})

display(widgets.VBox([slider_x_0, slider_y_0, slider_theta_0, slider_x_ref, slider_y_ref, interactive_plot]))


# Exercise 1h: MPC Formulation

### Discretization Schemes

Recall the four discretization schemes learned in the lecture (Euler Forwards, Euler Backwards, Trapezoidal, Exact). Implement them below:

In [7]:
def discretize_euler_forward(A: np.ndarray, B: np.ndarray, C: np.ndarray, T: float) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    A_d = np.eye(A.shape[0]) + T * A
    B_d = T * B
    C_d = C
    return A_d, B_d, C_d

def discretize_euler_backward(A: np.ndarray, B: np.ndarray, C: np.ndarray, T: float) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    A_d = np.linalg.inv(np.eye(A.shape[0]) - T * A)
    B_d = T * A_d @ B
    C_d = C
    return A_d, B_d, C_d

def discretize_trapezoidal(A: np.ndarray, B: np.ndarray, C: np.ndarray, h: float) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    I = np.eye(A.shape[0])
    A_d = np.linalg.inv(I - 0.5 * h * A) @ (I + 0.5 * h * A)
    B_d = np.linalg.inv(I - 0.5 * h * A) @ (h * B)
    C_d = C.copy() if C is not None else None
    return A_d, B_d, C_d

def discretize_exact(A: np.ndarray, B: np.ndarray, C: np.ndarray, h: float) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    n = A.shape[0]
    m = B.shape[1]
    M = np.zeros((n + m, n + m))
    M[:n, :n] = A
    M[:n, n:] = B
    expM = expm(h * M)
    Ad = expM[:n, :n]
    Bd = expM[:n, n:]
    Cd = C.copy() if C is not None else None
    return Ad, Bd, Cd

### Reference-Tracking MPC Formulation with State Constraints

Implement the reference tracking formulation determined in Exercise 1g below and simulate the system.

In [ ]:
methods = {
    'Euler Forward': discretize_euler_forward,
    'Euler Backward': discretize_euler_backward,
    'Trapezoidal':    discretize_trapezoidal,
    'Exact':          discretize_exact
}

# ——— Choose which discretization to use ———
disc_method = 'Exact'

nx     = A.shape[0]     # number of states
nu     = B.shape[1]     # number of inputs
T_sim  = 100            # number of discrete‐time steps per simulation
dt_def = 0.1            # default dt for computing terminal cost

# ——— Precompute terminal cost P_inf (for MPC) at the default dt ———
Ad, Bd, _ = methods[disc_method](A, B, C, dt_def)
P_inf = solve_discrete_are(Ad, Bd, Q, R)

def simulate_dlqr(x_0: float, y_0: float, theta_0: float, x_ref: float, y_ref: float, dt_dt: float) -> Tuple[np.ndarray, np.ndarray]:
    P_inf = solve_discrete_are(Ad, Bd, Q, R)
    K_d = np.linalg.inv(R + Bd.T @ P_inf @ Bd) @ (Bd.T @ P_inf @ Ad)

    ref = np.array([x_ref, 0, y_ref, 0, 0, 0])
    hover_vec = np.array([m*g/2, m*g/2])

    x = np.zeros((nx, T_sim + 1))
    x[:, 0] = [x_0, 0, y_0, 0, theta_0, 0]
    for k in range(T_sim):
        err = x[:, k] - ref
        u = np.clip(-K_d @ err + hover_vec, 0, None)
        x[:, k+1] = Ad @ x[:, k] + Bd @ (u - hover_vec)

    t = np.linspace(0, T_sim*dt_dt, T_sim+1)
    return x, t

def solve_mpc(x_k: np.ndarray, x_ref: float, y_ref: float, N: int) -> np.ndarray:
    # Initialize optimization variables
    x = cp.Variable((nx, N+1))
    u = cp.Variable((nu, N))

    # Initialize reference
    ref = np.array([x_ref, 0, y_ref, 0, 0, 0])

    # Initialize cost function and constraints
    cost = 0
    constraints = [x[:, 0] == x_k]

    for k in range(N):
        cost += cp.quad_form(x[:, k] - ref, Q) + cp.quad_form(u[:, k], R)
        constraints += [x[:, k+1] == Ad @ x[:, k] + Bd @ u[:, k]]
        constraints += [x[4, k] >= -0.2, x[4, k] <= 0.2]

    cost += cp.quad_form(x[:, N] - ref, P_inf)

    prob = cp.Problem(cp.Minimize(cost), constraints)
    prob.solve(solver=cp.OSQP, verbose=False)
    return u[:, 0].value.flatten()

def simulate_mpc(x_0: float, y_0: float, theta_0: float, x_ref: float, y_ref:float, dt_dt: float, N: int) -> Tuple[np.ndarray, np.ndarray]:
    ref = np.array([x_ref, 0, y_ref, 0, 0, 0])
    hover_vec = np.array([m*g/2, m*g/2])

    x = np.zeros((nx, T_sim+1))
    x[:, 0] = [x_0, 0, y_0, 0, theta_0, 0]

    for k in range(T_sim):
        u0 = solve_mpc(x[:, k], x_ref, y_ref, N)
        u = np.clip(u0 + hover_vec, 0, None)
        x[:, k+1] = Ad @ x[:, k] + Bd @ (u - hover_vec)

    t = np.linspace(0, (T_sim+1)*dt_dt, T_sim+1)
    return x, t

def simulate_compare(x_0: float, y_0: float, theta_0: float,
                     x_ref: float, y_ref: float, N: int, dt_dt: float = 0.1) -> None:
    # Run both controllers
    x_lqr, t = simulate_dlqr(x_0, y_0, theta_0, x_ref, y_ref, dt_dt)
    x_mpc, _ = simulate_mpc(x_0, y_0, theta_0, x_ref, y_ref, dt_dt, N)

    # Prepare plotting info
    state_idxs  = [0, 2, 4]
    labels      = {0: 'X Position (m)', 2: 'Y Position (m)', 4: r'Pitch $\theta$ (rad)'}
    refs        = {0: x_ref,      2: y_ref,      4: 0}
    ylims       = {0: (-5, 5),     2: (-5, 5),     4: (-0.5*np.pi, 0.5*np.pi)}

    fig, axes = plt.subplots(len(state_idxs), 1, figsize=(8, 9), sharex=True)

    for ax, idx in zip(axes, state_idxs):
        # plot trajectories
        ax.plot(t, x_lqr[idx],  '-', label='DLQR')
        ax.plot(t, x_mpc[idx], '--', label=f'DMPC (N={N})')

        # reference line
        ax.axhline(refs[idx], color='k', linestyle=':', label='ref')

        # special: θ constraints
        if idx == 4:
            ax.axhline(-0.2, color='r', linestyle='--', label='constraint')
            ax.axhline( 0.2, color='r', linestyle='--')

        # axis formatting
        ax.set_ylabel(labels[idx])
        ax.set_ylim(ylims[idx])
        ax.legend(loc='best')

    axes[-1].set_xlabel('Time (s)')
    plt.tight_layout()
    plt.show()

# Widgets
slider_x_0 = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='x_0')
slider_y_0 = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='y_0')
slider_theta_0 = widgets.FloatSlider(min=-0.5*np.pi, max=0.5*np.pi, step=0.1, value=0.0, description='theta_0')
slider_x_ref = widgets.FloatSlider(min=-5, max=5, step=0.1, value=-2.0, description='x_ref')
slider_y_ref = widgets.FloatSlider(min=-5, max=5, step=0.1, value=4.0, description='y_ref')
slider_N = widgets.IntSlider(min=1, max=20, step=1, value=5, description='N (MPC horizon)')

interactive_plot = widgets.interactive_output(simulate_compare, {
    'x_0': slider_x_0,
    'y_0': slider_y_0,
    'theta_0': slider_theta_0,
    'x_ref': slider_x_ref,
    'y_ref': slider_y_ref,
    'N': slider_N
})

display(widgets.VBox([slider_x_0, slider_y_0, slider_theta_0, slider_x_ref, slider_y_ref, slider_N, interactive_plot]))


# Linear MPC for Nonlinear Drone

In [9]:
A = np.array([[0,1,0,0,0,0],
            [0,0,0,0,g,0],
            [0,0,0,1,0,0],
            [0,0,0,0,0,0],
            [0,0,0,0,0,1],
            [0,0,0,0,0,0]])
B = np.array([[0,0],
            [0,0],
            [0,0],
            [1/m,1/m],
            [0,0],
            [l/I_z,-l/I_z]])
C = np.array([[1,0,0,0,0,0],
            [0,0,1,0,0,0],
            [0,0,0,0,1,0]])
# Process Noise
Q_lqe = np.diag([0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
# Measurement Noise
R_lqe = np.diag([0.01, 0.01, 0.01])
# Cost on State
Q_lqr = np.diag([10, 1, 10, 1, 100, 10])
# Cost on Input
R_lqr = np.eye(2)

@njit
def nonlinear_dynamics(t: float, state: np.ndarray, u: np.ndarray):
    x, x_dot, y, y_dot, theta, theta_dot = state
    T_1, T_2 = u + np.array([m*g/2, m*g/2])

    x_dd = (T_1 + T_2) / m * np.sin(theta)
    y_dd = (T_1 + T_2) / m * np.cos(theta) - g
    theta_dd = (T_1 - T_2) * l / I_z

    return np.array([x_dot, x_dd, y_dot, y_dd, theta_dot, theta_dd])

def lqe(Q_lqe: np.ndarray, R_lqe: np.ndarray) -> np.ndarray:
    P = solve_continuous_are(A.T, C.T, Q_lqe, R_lqe)
    L = P @ C.T @ np.linalg.inv(R_lqe)
    return L

def lqr(Q_lqr: np.ndarray, R_lqr: np.ndarray) -> np.ndarray:
    P = solve_continuous_are(A, B, Q_lqr, R_lqr)
    K = np.linalg.inv(R_lqr) @ B.T @ P
    return K

def linear_dynamics(t: float, state_hat: np.ndarray, u: np.ndarray, y: np.ndarray, L: np.ndarray):
    state_hat_dot = A @ state_hat + B @ u + L @ (y - C @ state_hat)
    x_dot, x_dd, y_dot, y_dd, theta_dot, theta_dd = state_hat_dot

    return np.array([x_dot, x_dd, y_dot, y_dd, theta_dot, theta_dd])

def sensor(t: float, state: np.ndarray) -> np.ndarray: 
    y = C @ state
    return y

@njit
def lqr_control(t: float, state_hat: np.ndarray, ref: np.ndarray, K: np.ndarray) -> np.ndarray:
    u = -K @ (state_hat - ref)
    return u


### Continuous-Time LQG

In [ ]:
def simulate_and_plot(x_0: np.ndarray, x_ref: np.ndarray, t_final: float = 10.0, dt: float = 0.05) -> None:
    # Solver Settings
    N = int(t_final / dt)
    
    # Set initial conditions
    state_0 = x_0
    state_hat_0 = x_0
    combined_state_0 = np.concatenate((state_0, state_hat_0))
    
    L = lqe(Q_lqe, R_lqe)
    K = lqr(Q_lqr, R_lqr)

    # Precalculate noise
    process_noise = np.random.multivariate_normal(mean=np.zeros(6), cov=Q_lqe, size=N)
    sensor_noise = np.random.multivariate_normal(mean=np.zeros(3), cov=R_lqe, size=N)

    def combined_dynamics(t: float, combined_state: np.ndarray) -> np.ndarray:
        state = combined_state[:6]
        state_hat = combined_state[6:]
        idx = min(int(t / dt), N - 1)

        # Control law (state feedback on estimate
        u = lqr_control(t, state_hat, x_ref, K)

        # Evolve true nonlinear dynamics
        d_state = nonlinear_dynamics(t, state, u) + process_noise[idx]
        # Evolve linear dynamics with Kalman filter
        y = sensor(t, state) + sensor_noise[idx]
        d_state_hat = linear_dynamics(t, state_hat, u, y, L)

        return np.concatenate((d_state, d_state_hat))
    
    sol = solve_ivp(combined_dynamics, [0, t_final], combined_state_0, method='RK23', max_step=0.05)

    t = sol.t
    state_vector = sol.y[:6, :]
    state_hat_vector = sol.y[6:, :]

    # Plotting
    fig, axes = plt.subplots(3, 1, figsize=(8, 8), sharex=True)
    axes[0].plot(t, state_vector[0], label='x')
    axes[0].plot(t, state_hat_vector[0], label='x_hat')
    axes[0].axhline(x_ref[0], color='r', linestyle='--', label='x_ref')
    axes[0].set_ylabel('X Position (m)')
    axes[0].set_ylim(-5, 5)
    axes[0].legend()

    axes[1].plot(t, state_vector[2], label='y')
    axes[1].plot(t, state_hat_vector[2], label='y_hat')
    axes[1].axhline(x_ref[2], color='r', linestyle='--', label='y_ref')
    axes[1].set_ylabel('Y Position (m)')
    axes[1].set_ylim(-5, 5)
    axes[1].legend()

    axes[2].plot(t, state_vector[4], label='theta')
    axes[2].plot(t, state_hat_vector[4], label='theta_hat')
    axes[2].axhline(x_ref[4], color='r', linestyle='--', label='theta_ref')
    axes[2].set_ylabel('Pitch θ (rad)')
    axes[2].set_ylim(-0.5*np.pi, 0.5*np.pi)
    axes[2].legend()

    plt.tight_layout()
    plt.show()

# Widgets
slider_x_0 = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='x_0')
slider_y_0 = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='y_0')
slider_theta_0 = widgets.FloatSlider(min=-0.5*np.pi, max=0.5*np.pi, step=0.1, value=0.0, description='theta_0')
slider_x_ref = widgets.FloatSlider(min=-5, max=5, step=0.1, value=-2.0, description='x_ref')
slider_y_ref = widgets.FloatSlider(min=-5, max=5, step=0.1, value=4.0, description='y_ref')

def wrapper(x_0: float, y_0: float, theta_0: float, x_ref: float, y_ref: float) -> None:
    x_0 = np.array([x_0, 0, y_0, 0, theta_0, 0])
    x_ref = np.array([x_ref, 0, y_ref, 0, 0, 0])
    simulate_and_plot(x_0, x_ref)

interactive_plot = widgets.interactive_output(wrapper, {
    'x_0': slider_x_0,
    'y_0': slider_y_0,
    'theta_0': slider_theta_0,
    'x_ref': slider_x_ref,
    'y_ref': slider_y_ref
})

display(widgets.VBox([slider_x_0, slider_y_0, slider_theta_0, slider_x_ref, slider_y_ref, interactive_plot]))

### Discrete-Time LQG

In [11]:
A = np.array([[0,1,0,0,0,0],
            [0,0,0,0,g,0],
            [0,0,0,1,0,0],
            [0,0,0,0,0,0],
            [0,0,0,0,0,1],
            [0,0,0,0,0,0]])
B = np.array([[0,0],
            [0,0],
            [0,0],
            [1/m,1/m],
            [0,0],
            [l/I_z,-l/I_z]])
C = np.array([[1,0,0,0,0,0],
            [0,0,1,0,0,0],
            [0,0,0,0,1,0]])

# Choose timestep
dt = 0.1
# Choose discretization method
Ad, Bd, Cd = discretize_exact(A, B, C, dt)

# Process Noise
Q_lqe = np.diag([0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
# Measurement Noise
R_lqe = np.diag([0.01, 0.01, 0.01])
# Cost on State
Q_lqr = np.diag([10, 1, 10, 1, 100, 10])
# Cost on Input
R_lqr = np.eye(2)

@njit
def nonlinear_dynamics(t: float, state: np.ndarray, u: np.ndarray):
    x, x_dot, y, y_dot, theta, theta_dot = state
    T_1, T_2 = u + np.array([m*g/2, m*g/2])

    x_dd = (T_1 + T_2) / m * np.sin(theta)
    y_dd = (T_1 + T_2) / m * np.cos(theta) - g
    theta_dd = (T_1 - T_2) * l / I_z

    return np.array([x_dot, x_dd, y_dot, y_dd, theta_dot, theta_dd])

def dlqe(Q_lqe: np.ndarray, R_lqe: np.ndarray) -> np.ndarray:
    P = solve_discrete_are(Ad.T, Cd.T, Q_lqe, R_lqe)
    S = Cd @ P @ Cd.T + R_lqe
    Ld = P @ Cd.T @ np.linalg.inv(S)
    return Ld

def dlqr(Q_lqr: np.ndarray, R_lqr: np.ndarray) -> np.ndarray:
    P = solve_discrete_are(Ad, Bd, Q_lqr, R_lqr)
    Kd = np.linalg.inv(R_lqr + Bd.T @ P @ Bd) @ (Bd.T @ P @ Ad)
    return Kd

def linear_dynamics(t: float, state_hat: np.ndarray, u: np.ndarray, y: np.ndarray, Ld: np.ndarray):
    state_hat = Ad @ state_hat + Bd @ u + Ld @ (y - Cd @ state_hat)
    return state_hat

def sensor(t: float, state: np.ndarray) -> np.ndarray: 
    y = Cd @ state
    return y

@njit
def lqr_control(t: float, state_hat: np.ndarray, ref: np.ndarray, K: np.ndarray) -> np.ndarray:
    u = -K @ (state_hat - ref)
    return u


In [ ]:
def simulate_and_plot(x_0: np.ndarray, x_ref: np.ndarray, t_final: float = 10.0, dt: float = 0.1) -> None:
    # Discretize once
    Ad, Bd, Cd = discretize_exact(A, B, C, dt)

    # Time vector
    N = int(np.ceil(t_final / dt))
    t = np.linspace(0, N * dt, N + 1)

    # Set initial conditions
    state = x_0
    state_hat = x_0

    # Storage
    sol = np.zeros((12, N + 1))
    sol[:, 0] = np.concatenate((state, state_hat))
    
    Ld = dlqe(Q_lqe, R_lqe)
    Kd = dlqr(Q_lqr, R_lqr)

    # Precalculate noise
    process_noise = np.random.multivariate_normal(mean=np.zeros(6), cov=Q_lqe, size=N)
    sensor_noise = np.random.multivariate_normal(mean=np.zeros(3), cov=R_lqe, size=N)

    for k in range(N):
        tk = t[k]
        u = lqr_control(tk, state_hat, x_ref, Kd)
        
        # Nonlinear dynamics via RK23 propagate
        rk_sol = solve_ivp(lambda tau, s: nonlinear_dynamics(tk, s, u), [tk, tk + dt], state, method='RK23', max_step=dt, t_eval=[tk + dt])
        state = rk_sol.y[:, -1] + 0.2*process_noise[k]

        y = sensor(tk, state) + 0.2*sensor_noise[k]
        state_hat = linear_dynamics(tk, state_hat, u, y, Ld)

        sol[:, k+1] = np.concatenate((state, state_hat))


    state_vector = sol[:6, :]
    state_hat_vector = sol[6:, :]

    # Plotting
    fig, axes = plt.subplots(3, 1, figsize=(8, 8), sharex=True)
    axes[0].plot(t, state_vector[0], label='x')
    axes[0].plot(t, state_hat_vector[0], label='x_hat')
    axes[0].axhline(x_ref[0], color='r', linestyle='--', label='x_ref')
    axes[0].set_ylabel('X Position (m)')
    axes[0].set_ylim(-5, 5)
    axes[0].legend()

    axes[1].plot(t, state_vector[2], label='y')
    axes[1].plot(t, state_hat_vector[2], label='y_hat')
    axes[1].axhline(x_ref[2], color='r', linestyle='--', label='y_ref')
    axes[1].set_ylabel('Y Position (m)')
    axes[1].set_ylim(-5, 5)
    axes[1].legend()

    axes[2].plot(t, state_vector[4], label='theta')
    axes[2].plot(t, state_hat_vector[4], label='theta_hat')
    axes[2].axhline(x_ref[4], color='r', linestyle='--', label='theta_ref')
    axes[2].set_ylabel('Pitch θ (rad)')
    axes[2].set_ylim(-0.5*np.pi, 0.5*np.pi)
    axes[2].legend()

    plt.tight_layout()
    plt.show()

# Widgets
slider_x_0 = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='x_0')
slider_y_0 = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='y_0')
slider_theta_0 = widgets.FloatSlider(min=-0.5*np.pi, max=0.5*np.pi, step=0.1, value=0.0, description='theta_0')
slider_x_ref = widgets.FloatSlider(min=-5, max=5, step=0.1, value=-2.0, description='x_ref')
slider_y_ref = widgets.FloatSlider(min=-5, max=5, step=0.1, value=4.0, description='y_ref')

def wrapper(x_0: float, y_0: float, theta_0: float, x_ref: float, y_ref: float) -> None:
    x_0 = np.array([x_0, 0, y_0, 0, theta_0, 0])
    x_ref = np.array([x_ref, 0, y_ref, 0, 0, 0])
    simulate_and_plot(x_0, x_ref)

interactive_plot = widgets.interactive_output(wrapper, {
    'x_0': slider_x_0,
    'y_0': slider_y_0,
    'theta_0': slider_theta_0,
    'x_ref': slider_x_ref,
    'y_ref': slider_y_ref
})

display(widgets.VBox([slider_x_0, slider_y_0, slider_theta_0, slider_x_ref, slider_y_ref, interactive_plot]))

### DMPC Formulation

In [13]:
def dmpc_control(state_hat_k: np.ndarray, x_ref: np.ndarray, N: int, P_inf: np.ndarray) -> np.ndarray:
    nx = A.shape[0]
    nu = B.shape[1]
    x = cp.Variable((nx, N+1))
    u = cp.Variable((nu, N))
    u_ref = np.array([m*g/2, m*g/2])

    cost = 0
    constraints = [x[:, 0] == state_hat_k]

    for k in range(N):
        cost += cp.quad_form(x[:, k] - x_ref, Q_lqr) + cp.quad_form(u[:, k] - u_ref, R_lqr)
        constraints += [x[:, k+1] == Ad @ x[:, k] + Bd @ u[:, k]]
        constraints += [x[4, k] >= -0.4, x[4, k] <= 0.4]

    cost += cp.quad_form(x[:, N] - x_ref, P_inf)
    prob = cp.Problem(cp.Minimize(cost), constraints)
    prob.solve(solver=cp.OSQP, verbose=False)
    return u[:, 0].value.flatten()

In [14]:
def simulate_and_plot(x_0: np.ndarray, x_ref: np.ndarray, N_prediction: int, t_final: float = 10.0, dt: float = 0.1) -> None:
    # Discretize once
    Ad, Bd, Cd = discretize_exact(A, B, C, dt)

    # Time vector
    N = int(np.ceil(t_final / dt))
    t = np.linspace(0, N * dt, N + 1)

    # Set initial conditions
    state = x_0
    state_hat = x_0

    # Storage
    sol = np.zeros((12, N + 1))
    sol[:, 0] = np.concatenate((state, state_hat))
    
    Ld = dlqe(Q_lqe, R_lqe)
    Kd = dlqr(Q_lqr, R_lqr)
    P_inf = solve_discrete_are(Ad, Bd, Q_lqr, R_lqr)

    # Precalculate noise
    process_noise = np.random.multivariate_normal(mean=np.zeros(6), cov=Q_lqe, size=N)
    sensor_noise = np.random.multivariate_normal(mean=np.zeros(3), cov=R_lqe, size=N)

    for k in range(N):
        tk = t[k]
        u = dmpc_control(state_hat, x_ref, N_prediction, P_inf)
        
        # Nonlinear dynamics via RK23 propagate
        rk_sol = solve_ivp(lambda tau, s: nonlinear_dynamics(tk, s, u), [tk, tk + dt], state, method='RK23', max_step=dt, t_eval=[tk + dt])
        state = rk_sol.y[:, -1] + 0.1*process_noise[k]

        y = sensor(tk, state) + 0.1*sensor_noise[k]
        state_hat = linear_dynamics(tk, state_hat, u, y, Ld)

        sol[:, k+1] = np.concatenate((state, state_hat))


    state_vector = sol[:6, :]
    state_hat_vector = sol[6:, :]

    # Plotting
    fig, axes = plt.subplots(3, 1, figsize=(8, 8), sharex=True)
    axes[0].plot(t, state_vector[0], label='x')
    axes[0].plot(t, state_hat_vector[0], label='x_hat')
    axes[0].axhline(x_ref[0], color='r', linestyle='--', label='x_ref')
    axes[0].set_ylabel('X Position (m)')
    axes[0].set_ylim(-5, 5)
    axes[0].legend()

    axes[1].plot(t, state_vector[2], label='y')
    axes[1].plot(t, state_hat_vector[2], label='y_hat')
    axes[1].axhline(x_ref[2], color='r', linestyle='--', label='y_ref')
    axes[1].set_ylabel('Y Position (m)')
    axes[1].set_ylim(-5, 5)
    axes[1].legend()

    axes[2].plot(t, state_vector[4], label='theta')
    axes[2].plot(t, state_hat_vector[4], label='theta_hat')
    axes[2].axhline(x_ref[4], color='r', linestyle='--', label='theta_ref')
    axes[2].set_ylabel('Pitch θ (rad)')
    axes[2].set_ylim(-0.5*np.pi, 0.5*np.pi)
    axes[2].legend()

    plt.tight_layout()
    plt.show()

# Widgets
slider_x_0 = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='x_0')
slider_y_0 = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='y_0')
slider_theta_0 = widgets.FloatSlider(min=-0.199, max=0.199, step=0.1, value=0.0, description='theta_0')
slider_x_ref = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='x_ref')
slider_y_ref = widgets.FloatSlider(min=-5, max=5, step=0.1, value=4.0, description='y_ref')
slider_N_prediction = widgets.IntSlider(min=1, max=50, step=1, value=5, description='N (MPC horizon)')

def wrapper(x_0: float, y_0: float, theta_0: float, x_ref: float, y_ref: float, N_prediction: int) -> None:
    x_0 = np.array([x_0, 0, y_0, 0, theta_0, 0])
    x_ref = np.array([x_ref, 0, y_ref, 0, 0, 0])
    simulate_and_plot(x_0, x_ref, N_prediction)

interactive_plot = widgets.interactive_output(wrapper, {
    'x_0': slider_x_0,
    'y_0': slider_y_0,
    'theta_0': slider_theta_0,
    'x_ref': slider_x_ref,
    'y_ref': slider_y_ref,
    'N_prediction': slider_N_prediction
})

display(widgets.VBox([slider_x_0, slider_y_0, slider_theta_0, slider_x_ref, slider_y_ref, slider_N_prediction, interactive_plot]))

### LQG, DLQG, MPC, MPC with soft constraints

System Dynamics

In [15]:
A = np.array([[0,1,0,0,0,0],
            [0,0,0,0,g,0],
            [0,0,0,1,0,0],
            [0,0,0,0,0,0],
            [0,0,0,0,0,1],
            [0,0,0,0,0,0]])
B = np.array([[0,0],
            [0,0],
            [0,0],
            [1/m,1/m],
            [0,0],
            [l/I_z,-l/I_z]])
C = np.array([[1,0,0,0,0,0],
            [0,0,1,0,0,0],
            [0,0,0,0,1,0]])
# Process Noise
Q_lqe = np.diag([0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
# Measurement Noise
R_lqe = np.diag([0.01, 0.01, 0.01])
# Cost on State
Q_lqr = np.diag([10, 1, 10, 1, 100, 10])
# Cost on Input
R_lqr = np.eye(2)

dt = 0.05
# Choose discretization method
Ad, Bd, Cd = discretize_exact(A, B, C, dt)

@njit
def nonlinear_dynamics(t: float, state: np.ndarray, u: np.ndarray):
    x, x_dot, y, y_dot, theta, theta_dot = state
    T_1, T_2 = u + np.array([m*g/2, m*g/2])

    x_dd = (T_1 + T_2) / m * np.sin(theta)
    y_dd = (T_1 + T_2) / m * np.cos(theta) - g
    theta_dd = (T_1 - T_2) * l / I_z

    return np.array([x_dot, x_dd, y_dot, y_dd, theta_dot, theta_dd])

def lqe(Q_lqe: np.ndarray, R_lqe: np.ndarray) -> np.ndarray:
    P = solve_continuous_are(A.T, C.T, Q_lqe, R_lqe)
    L = P @ C.T @ np.linalg.inv(R_lqe)
    return L

def lqr(Q_lqr: np.ndarray, R_lqr: np.ndarray) -> np.ndarray:
    P = solve_continuous_are(A, B, Q_lqr, R_lqr)
    K = np.linalg.inv(R_lqr) @ B.T @ P
    return K

def dlqe(Q_lqe: np.ndarray, R_lqe: np.ndarray) -> np.ndarray:
    P = solve_discrete_are(Ad.T, Cd.T, Q_lqe, R_lqe)
    S = Cd @ P @ Cd.T + R_lqe
    Ld = P @ Cd.T @ np.linalg.inv(S)
    return Ld

def dlqr(Q_lqr: np.ndarray, R_lqr: np.ndarray) -> np.ndarray:
    P = solve_discrete_are(Ad, Bd, Q_lqr, R_lqr)
    Kd = np.linalg.inv(R_lqr + Bd.T @ P @ Bd) @ (Bd.T @ P @ Ad)
    return Kd

def linear_dynamics(t: float, state_hat: np.ndarray, u: np.ndarray, y: np.ndarray, L: np.ndarray):
    state_hat_dot = A @ state_hat + B @ u + L @ (y - C @ state_hat)
    x_dot, x_dd, y_dot, y_dd, theta_dot, theta_dd = state_hat_dot

    return np.array([x_dot, x_dd, y_dot, y_dd, theta_dot, theta_dd])

def discrete_linear_dynamics(t: float, state_hat: np.ndarray, u: np.ndarray, y: np.ndarray, Ld: np.ndarray):
    state_hat = Ad @ state_hat + Bd @ u + Ld @ (y - Cd @ state_hat)
    return state_hat

def sensor(t: float, state: np.ndarray) -> np.ndarray: 
    y = C @ state
    return y


Controllers:

In [33]:
@njit
def lqr_control(t: float, state_hat: np.ndarray, ref: np.ndarray, K: np.ndarray) -> np.ndarray:
    u = -K @ (state_hat - ref)
    return u

def dmpc_control(state_hat_k: np.ndarray, x_ref: np.ndarray, N: int, P_inf: np.ndarray, soft: bool) -> np.ndarray:
    nx = A.shape[0]
    nu = B.shape[1]
    x = cp.Variable((nx, N+1))
    u = cp.Variable((nu, N))
    u_ref = np.array([m*g/2, m*g/2])
    if soft:
        rho_soft = 1000.0
        s = cp.Variable(N, nonneg=True)

    cost = 0
    constraints = [x[:, 0] == state_hat_k]

    for k in range(N):
        cost += cp.quad_form(x[:, k] - x_ref, Q_lqr) + cp.quad_form(u[:, k] - u_ref, R_lqr)
        constraints += [x[:, k+1] == Ad @ x[:, k] + Bd @ u[:, k]]
        if soft:
            constraints += [x[4, k] <= 0.4 + s[k]]
            constraints += [x[4, k] >= -0.4 - s[k]]
            cost += rho_soft * s[k]
        else:
            constraints += [x[4, k] >= -0.4, x[4, k] <= 0.4]

    cost += cp.quad_form(x[:, N] - x_ref, P_inf)
    prob = cp.Problem(cp.Minimize(cost), constraints)
    prob.solve(solver=cp.OSQP, verbose=False)
    return u[:, 0].value.flatten()

Simulate all:

In [36]:
def run_lqg(x_0: np.ndarray, x_ref: np.ndarray, t_final: float = 10.0, dt: float = 0.05) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    N = int(t_final / dt)
    t = np.linspace(0, N * dt, N + 1)
    
    # Set initial conditions
    state_0 = x_0.copy()
    state_hat_0 = x_0.copy()
    combined_state_0 = np.concatenate((state_0, state_hat_0))
    u_hist = np.zeros((2, N))
    
    L = lqe(Q_lqe, R_lqe)
    K = lqr(Q_lqr, R_lqr)

    # Precalculate noise
    process_noise = np.random.multivariate_normal(mean=np.zeros(6), cov=Q_lqe, size=N)
    sensor_noise = np.random.multivariate_normal(mean=np.zeros(3), cov=R_lqe, size=N)

    def combined_dynamics(ti: float, combined_state: np.ndarray) -> np.ndarray:
        state = combined_state[:6]
        state_hat = combined_state[6:]
        idx = min(int(ti / dt), N - 1)

        # Control law (state feedback on estimate
        u = lqr_control(ti, state_hat, x_ref, K)
        u_hist[:, idx] = u

        # Evolve true nonlinear dynamics
        d_state = nonlinear_dynamics(ti, state, u) + process_noise[idx]
        # Evolve linear dynamics with Kalman filter
        y = sensor(ti, state) + sensor_noise[idx]
        d_state_hat = linear_dynamics(ti, state_hat, u, y, L)

        return np.concatenate((d_state, d_state_hat))
    
    sol = solve_ivp(combined_dynamics, [0, t_final], combined_state_0, method='RK23', max_step=dt, t_eval=t)

    t = sol.t
    state_vector = sol.y[:6, :]
    state_hat_vector = sol.y[6:, :]

    return t, state_vector, state_hat_vector, u_hist

def run_dlqg(x_0: np.ndarray, x_ref: np.ndarray, t_final: float = 10.0, dt: float = 0.05)  -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    Ad, Bd, Cd = discretize_exact(A, B, C, dt)
    N = int(np.ceil(t_final / dt))
    t = np.linspace(0, N * dt, N + 1)

    # Set initial conditions
    state = x_0.copy()
    state_hat = x_0.copy()

    # Storage
    sol = np.zeros((12, N + 1))
    sol[:, 0] = np.concatenate((state, state_hat))
    u_hist = np.zeros((2, N))
    
    Ld = dlqe(Q_lqe, R_lqe)
    Kd = dlqr(Q_lqr, R_lqr)

    # Precalculate noise
    process_noise = np.random.multivariate_normal(mean=np.zeros(6), cov=Q_lqe, size=N)
    sensor_noise = np.random.multivariate_normal(mean=np.zeros(3), cov=R_lqe, size=N)

    for k in tqdm(range(N), desc="DLQG Progress", unit="step"):
        tk = t[k]
        u = lqr_control(tk, state_hat, x_ref, Kd)
        u_hist[:, k] = u
        
        # Nonlinear dynamics via RK23 propagate
        rk_sol = solve_ivp(lambda tau, s: nonlinear_dynamics(tk, s, u), [tk, tk + dt], state, method='RK23', max_step=dt, t_eval=[tk + dt])
        state = rk_sol.y[:, -1] + 0.2*process_noise[k]

        y = sensor(tk, state) + 0.2*sensor_noise[k]
        state_hat = discrete_linear_dynamics(tk, state_hat, u, y, Ld)

        sol[:, k+1] = np.concatenate((state, state_hat))

    state_vector = sol[:6, :]
    state_hat_vector = sol[6:, :]
    return t, state_vector, state_hat_vector, u_hist

def run_mpc(x_0: np.ndarray, x_ref: np.ndarray, N_prediction: int, soft: bool = False, t_final: float = 10.0, dt: float = 0.1) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    Ad, Bd, Cd = discretize_exact(A, B, C, dt)

    # Time vector
    N = int(np.ceil(t_final / dt))
    t = np.linspace(0, N * dt, N + 1)

    # Set initial conditions
    state = x_0.copy()
    state_hat = x_0.copy()

    # Storage
    sol = np.zeros((12, N + 1))
    sol[:, 0] = np.concatenate((state, state_hat))
    u_hist = np.zeros((2, N))
    
    Ld = dlqe(Q_lqe, R_lqe)
    Kd = dlqr(Q_lqr, R_lqr)
    P_inf = solve_discrete_are(Ad, Bd, Q_lqr, R_lqr)

    # Precalculate noise
    process_noise = np.random.multivariate_normal(mean=np.zeros(6), cov=Q_lqe, size=N)
    sensor_noise = np.random.multivariate_normal(mean=np.zeros(3), cov=R_lqe, size=N)

    if soft:
        desc = "Soft MPC Progress"
    else:
        desc = "MPC Progress"

    for k in tqdm(range(N), desc=desc, unit="step"):
        tk = t[k]
        u = dmpc_control(state_hat, x_ref, N_prediction, P_inf, soft)
        u_hist[:, k] = u
        
        # Nonlinear dynamics via RK23 propagate
        rk_sol = solve_ivp(lambda tau, s: nonlinear_dynamics(tk, s, u), [tk, tk + dt], state, method='RK23', max_step=dt, t_eval=[tk + dt])
        state = rk_sol.y[:, -1] + 0.1*process_noise[k]

        y = sensor(tk, state) + 0.1*sensor_noise[k]
        state_hat = discrete_linear_dynamics(tk, state_hat, u, y, Ld)

        sol[:, k+1] = np.concatenate((state, state_hat))


    state_vector = sol[:6, :]
    state_hat_vector = sol[6:, :]

    return t, state_vector, state_hat_vector, u_hist



In [38]:
def simulate_and_plot(x_0: np.ndarray,
                       x_ref: np.ndarray,
                       N_prediction: int = 5,
                       t_final: float = 10.0,
                       dt: float = 0.05) -> None:
    # Run each controller
    lqg_t,  lqg_x,  lqg_xhat,  lqg_u     = run_lqg(x_0, x_ref, t_final, dt)
    dlqg_t, dlqg_x, dlqg_xhat, dlqg_u    = run_dlqg(x_0, x_ref, t_final, dt)
    try:
        mpc_t,  mpc_x,  mpc_xhat,  mpc_u     = run_mpc(x_0, x_ref, N_prediction, False, t_final, dt)
        mpc_infeasile = False
    except cp.error.SolverError:
        mpc_t = np.zeros(0)
        mpc_x = np.zeros((6, 0))
        mpc_xhat = np.zeros((6, 0))
        mpc_u = np.zeros((2, 0))
        mpc_infeasile = True
    try:
        soft_t, soft_x, soft_xhat, soft_u    = run_mpc(x_0, x_ref, N_prediction, True,  t_final, dt)
        soft_infeasible = False
    except cp.error.SolverError:
        soft_t = np.zeros(0)
        soft_x = np.zeros((6, 0))
        soft_xhat = np.zeros((6, 0))
        soft_u = np.zeros((2, 0))
        soft_infeasible = True

    controllers = [
        ("LQG",     lqg_t,  lqg_x,   lqg_xhat,  lqg_u, False),
        ("DLQG",    dlqg_t, dlqg_x,  dlqg_xhat, dlqg_u, False),
        ("MPC",     mpc_t,  mpc_x,   mpc_xhat,  mpc_u, mpc_infeasile),
        ("Soft MPC",soft_t, soft_x,  soft_xhat, soft_u, soft_infeasible)
    ]

    fig, axes = plt.subplots(3, 4, figsize=(16, 9), sharex='col')

    # Physical constants (assumes these are in scope)
    # m: mass of the vehicle, g: gravitational acceleration
    # If not, define here, e.g., m = 1.0; g = 9.81
    for col, (name, t, x, xhat, u, infeasible) in enumerate(controllers):
        if infeasible:
                    for row in range(3):
                        ax = axes[row, col]
                        ax.text(0.5, 0.5, 'infeasible',
                                ha='center', va='center',
                                transform=ax.transAxes,
                                fontsize='large', color='red')
                        ax.set_xticks([])
                        ax.set_yticks([])
                    axes[0, col].set_title(name)
                    continue
        
        u_plot = u + m * g / 2

        # Row 1: x & y positions vs estimates
        ax = axes[0, col]
        ax.plot(t, x[0, :],    label='x',     color='darkblue')
        ax.plot(t, xhat[0, :], label='x_hat', linestyle='--', color='lightblue')
        ax.plot(t, x[2, :],    label='y',     color='darkgreen')
        ax.plot(t, xhat[2, :], label='y_hat', linestyle='--', color='lightgreen')
        # reference lines
        ax.axhline(x_ref[0], linestyle=':', color='grey')
        ax.axhline(x_ref[2], linestyle=':', color='grey')
        ax.set_ylim(-5, 5)
        if col == 0:
            ax.set_ylabel('pos (m)')
        ax.set_title(name)
        ax.legend(loc='upper right', fontsize='small')

        # Row 2: pitch θ vs estimate
        ax = axes[1, col]
        ax.plot(t, x[4, :],    label='θ',      color='purple')
        ax.plot(t, xhat[4, :], label='θ_hat',  linestyle='--', color='violet')
        # theta reference
        ax.axhline(0,               linestyle=':', color='grey')
        ax.axhline(x_ref[4],       linestyle=':', color='grey')
        # hard/soft angle constraints dotted
        if name in ('MPC', 'Soft MPC'):
            ax.axhline( 0.4, linestyle=':', color='red')
            ax.axhline(-0.4, linestyle=':', color='red')
        ax.set_ylim(-1, 1)
        if col == 0:
            ax.set_ylabel('θ (rad)')
        ax.legend(loc='upper right', fontsize='small')

        # Row 3: control inputs u1, u2
        ax = axes[2, col]
        ax.plot(t[:-1], u_plot[0, :], label='u₁', linestyle='-', color='black')
        ax.plot(t[:-1], u_plot[1, :], label='u₂', linestyle='--', color='grey')
        if col == 0:
            ax.set_ylabel('thrust')
        ax.set_xlabel('time (s)')
        ax.legend(loc='upper right', fontsize='small')

    plt.tight_layout()
    plt.show()


# Widgets
slider_x_0 = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='x_0')
slider_y_0 = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='y_0')
slider_theta_0 = widgets.FloatSlider(min=-0.5*np.pi, max=0.5*np.pi, step=0.1, value=0.0, description='theta_0')
slider_x_ref = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='x_ref')
slider_y_ref = widgets.FloatSlider(min=-5, max=5, step=0.1, value=2.0, description='y_ref')
slider_N_prediction = widgets.IntSlider(min=1, max=50, step=1, value=5, description='N (MPC horizon)')

def wrapper(x_0: float, y_0: float, theta_0: float, x_ref: float, y_ref: float, N_prediction: int) -> None:
    x_0 = np.array([x_0, 0, y_0, 0, theta_0, 0])
    x_ref = np.array([x_ref, 0, y_ref, 0, 0, 0])
    simulate_and_plot(x_0, x_ref, N_prediction)

interactive_plot = widgets.interactive_output(wrapper, {
    'x_0': slider_x_0,
    'y_0': slider_y_0,
    'theta_0': slider_theta_0,
    'x_ref': slider_x_ref,
    'y_ref': slider_y_ref,
    'N_prediction': slider_N_prediction
})

display(widgets.VBox([slider_x_0, slider_y_0, slider_theta_0, slider_x_ref, slider_y_ref, slider_N_prediction, interactive_plot]))